# IMPORT

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from pandas_profiling import ProfileReport
from sklearn.pipeline import Pipeline
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
from preprocess import preprocess
from preprocess import normalize
from sklearn_pandas import DataFrameMapper
import sqlite3 as sq
import os
%matplotlib inline

In [ ]:
df = pd.read_csv('../data/profiles_revised.csv')
df.columns = df.columns.str.replace('\t', '') # needed?
origin = df.columns

In [ ]:
def print_col_values(list, filename):
    with open(r'{}.txt'.format(filename), 'w') as fp:
        for element in list:
            fp.write("{}\n".format(element))

In [ ]:
def df_distinct_values(df, folder):
    if not os.path.exists(folder):
        os.makedirs('./exploration/'+folder)
    for (index, colname) in enumerate(df.columns):
        #print(index, colname)
        distinc_values = df[colname].unique()
        print_col_values(list=distinc_values, filename='./exploration/{}/{}-{}-values'.format(folder, index, colname))

In [ ]:
df_distinct_values(df=df, folder='origin')

In [ ]:
#profile = ProfileReport(df, title='Pandas Profilign Report')
#profile.to_notebook_iframe()
#profile.to_file('./exploration/pandas_profiling_data_report.html')

# CLEAN

In [ ]:
#df = df[['age', 'body_type']]
df_clean = preprocess(df.columns, df)

In [ ]:
# TODO: fix '\'
#df_distinct_values(df=df_clean, folder='cleaned')

# SAVE

In [ ]:
df_clean.to_csv('./data/cleaned.csv')

In [ ]:
#table_name = 'okcupid_clean'
#table_path = './data/'+ table_name +'_db'
#conn = sq.connect('{}.sqlite'.format(table_path)) # creates file
#df_clean.to_sql(table_path, conn, if_exists='replace', index=False) # writes to file
#conn.close() # good practice: close connection

# STANDARDIZE

In [ ]:
df_clean.head()
sample = df_clean.iloc[:1]

In [ ]:
normalize(df)

In [ ]:
continuous_cols = ['age', 'height']
categorical_cols = ['body_type', 'drinks', 'drugs', 'income', 'job', 'orientation', 'sex', 'smokes',
'diet','diet_modifier',
'education_status', 'education_institution',
'offspring_status', 'offspring_future',
'pets_cats', 'pets_dogs',
'religion_type', 'religion_modifier',
'sign', 'sign_modifier']
ethnities_cols = df_clean[df_clean.columns[pd.Series(df_clean.columns).str.startswith('ethnicities')]].columns
speaks_cols = df_clean[df_clean.columns[pd.Series(df_clean.columns).str.startswith('speaks')]].columns

In [ ]:
# Example
mapper = DataFrameMapper([
  ('body_type', LabelEncoder()),
  (['age'], StandardScaler())],
  #[(categorical_col, LabelBinarizer()) for categorical_col in categorical_cols],
  df_out=True 
)
print(mapper)

In [ ]:
# Mapper for checking
mapper = DataFrameMapper([
  #('drinks', LabelEncoder()),
  (['age'], StandardScaler())] +
  [(categorical_col, LabelBinarizer()) for categorical_col in categorical_cols],
  df_out=True 
)
print(mapper)

In [ ]:
# Real mapper
mapper = DataFrameMapper(
  [([continuous_col], StandardScaler()) for continuous_col in continuous_cols] +
  [(categorical_col, LabelEncoder()) for categorical_col in categorical_cols] +
  [(ethnities_col, LabelEncoder()) for ethnities_col in ethnities_cols] +
  [(speaks_col, LabelEncoder()) for speaks_col in speaks_cols],
  df_out=True 
)
mapper

In [ ]:
df_std = np.round(mapper.fit_transform(df_clean.copy()),2)

In [ ]:
#table_name = 'okcupid_std'
#table_path = './data/'+ table_name +'_db'
#conn = sq.connect('{}.sqlite'.format(table_path)) # creates file
#std_df.to_sql(table_path, conn, if_exists='replace', index=False) # writes to file
#conn.close() # good practice: close connection

In [ ]:
table_names = ['okcupid_clean', 'okcupid_std']
#table_path = './data/'+ table_name +'_db'

dfs = {
    "std_clean": df_clean,
    "dt_std": df_std,
}

with sq.connect("okcupid.sqlite") as db:
    df_clean.to_sql('okcupid_clean', db, if_exists="replace", index=False)
    df_std.to_sql('okcupid_std', db, if_exists="replace", index=False)

In [ ]:
sample

In [ ]:
#sample = pd.DataFrame({'age': [30], 'body_type': ['fit']})
#np.round(mapper.transform(sample), 2)

In [ ]:
np.round(mapper.transform(sample), 2)

In [ ]:
# https://scikit-learn.org/stable/modules/preprocessing.html
# https://stackoverflow.com/questions/43554821/feature-preprocessing-of-both-continuous-and-categorical-variables-of-integer-t
# https://stackoverflow.com/questions/53152627/saving-standardscaler-model-for-use-on-new-datasets?noredirect=1&lq=1
# https://stackoverflow.com/questions/38780302/predicting-new-data-using-sklearn-after-standardizing-the-training-data

# RESULT

In [ ]:
plt.figure(figsize=(5,5))
sns.heatmap(df.corr(), center=0, annot=True)
plt.title('Correlation Map')
plt.show()
plt.savefig('./exploration/correlation-map.png')

In [ ]:
plt.figure(figsize=(5,5))
sns.pairplot(data=df, hue='lables', palette='RdBu')
plt.title('Correlation Map')
plt.show()
plt.savefig('./exploration/pairplot.png')